This notebook is for downloading the MACA v2 metdata, harmonizing the different data arrays with different resolutions, and coding thresholds for suitability ratings to each of the layers. Additionally, a fuzzly logic model would be applied to designate each cell on a 0-1 continuous suitability scale for plotting and identification of more- and less-suitable areas under different climate change scenarios. 

Because I have not yet gotten to these steps, this notebook contains only initial pseudocode. 

In [ ]:
import os
import pathlib
import re
import zipfile
from io import BytesIO
from glob import glob
import requests

import earthaccess
import earthpy as et
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
from math import floor, ceil

import xarray as xr
import rioxarray as rxr  #work with raster data
from rioxarray.merge import merge_arrays # Merge rasters
from rasterio.mask import mask
import xrspatial # 
import warnings

In [ ]:
# Maca v2 metdata download
# I would define a df or a dictionary with each combination of GCM, time period, and climate variable:
# The climate variables would be: Maximum temperature, mean temperature, and total precipitation
#
# I would select a single time period, 2060-2065. The GCM would be HadGEM2-ES365 from CMIP5. 
#

# Define the url template
maca_url_template = ( <format of filepath to .nc>

)

#Start an empty list for the urls
maca_url_list = []

# I would then accumulate all the data arrays covering 
# either of the forest gdfs. 
#for-loop to read in each .nc file
maca_da = xr.open_dataset(maca_url_template)
maca_url_list.append(maca_da)
maca_das

# Plot the data arrays
#maca_das.plot()

In [ ]:
# Convert MACA data to a data frame for aggregating


# Aggregation/resampling step 1: Calculate annual measures from the monthly data by resampling


# For example, compute averages for mean/max temperature and sum for precipitation

# Agg/resampling step 2: Average the annual estimates across the 5-year temporal range .

In [ ]:
# Necessary data checks go here. Review descriptive statistics for 
#
#
# Data visualization: Histograms for data checking

In [ ]:
# Data Harmonization phase: All raster data needs standardized


# Reproject the coarser raster data (MACA) to the rio.crs of the finer-scale data
# 

# Get all data arrays: SRTM, POLARIS, and MACA to an aligned raster grid
# 

# Combine the layers using rasterio


In [ ]:
# Define suitability thresholds for all variables

# Habitat suitability variables: Elevation, Soil pH, mean temp, max. temp, and 
#   total precipitation. 
#
# Elevation: 2,000-7,000 Ft
# Soil pH: ideal range 5.0 - 6.5
#
# Mean temperature: Average ann. max: 29C/84F; Avg ann mean: 7C/45F
# Max temp: Abs ann max: 41C/106F (Schmidt & Shearer)
# 
# Precipitation: Average ann. total precip: 710mm/28in up to 810mm in the southern part of its range. 
# Extreme precip range is between 460 mm (18in) and 1270mm (50 in)

In [ ]:
# Code and run a suitability model using fuzzy logic
# 
# I would divide the ranges of each of the habitat variables into scores where
# a high score is given to the middle part of the distributions where the current
# average/median conditions are. Scores would go down in increments as a given
# value got farther away on either side of the optimal range. 
# For example, looking at precipitation, I would perhaps group 25 - 33 inches of rain
# as the highest score, say 1.0. (I would allow more precipitation because this is generally
# a good thing for trees.) Then 22-24 in and 34-35 in of annual precip would be 0.8, and so on. 
# Very low values of annual precip (<18in) would be scored very low. 
#
# Multiply the scores for all the habitat variables for each cell

In [ ]:
# Plot the new raster layer with the suitability scores, ranging 0.0-1.0.
#
# I would probably use hvplot to get an interactive plot, and choose a 
# colormap that would reflect suitability, maybe green-to-orange or green 
# to brown scale, with green representing higher suitability. 
#
#

After running the fuzzy logic model and multiplying across the layers, I would plot the suitability maps for my two national forests. As well, I would interpret the spatial variation in the scores across each grassland. I would even consider having a third comparison layer, and that would be using back-casted climate estimates for a recent time period, say 2010-2015 if available. Then I would have a baseline suitability map for these two forests, and this would help to interpret how much change there was in the climate change projections for 2060-65 (fifty years later).


### References

Schmidt, W.C., Shearer, R.C. "Western Larch". United States Forest Service. Retrieved from: https://www.srs.fs.usda.gov/pubs/misc/ag_654/volume_1/larix/occidentalis.htm

Oregeon Forest Resources Institute. "Western Larch". Retrieved from: https://oregonforests.org/forest-types-tree-guide/tree-variety/western-larch.